In [2]:
# This should be roughly the content of the first code cell
import numpy as np
import random
np.random.seed(1337)
random.seed(1337)

# Plotting support
from matplotlib import pyplot as plt
# from plotnine import
# Standard libraries
import pandas as pd
import sklearn as sk
import tensorflow as tf
import time
import os
from skimage import io
import cv2
from scipy import stats
from skimage.util import compare_images
from PIL import Image
from skimage.metrics import mean_squared_error, structural_similarity, hausdorff_distance, normalized_root_mse, peak_signal_noise_ratio
plt.rc('image', cmap='gray')



In [3]:
table1 = np.load('training/feature_table1.npy')
table2 = np.load('training/feature_table2.npy')
table3 = np.load('training/feature_table3.npy')
table4 = np.load('training/feature_table4.npy')
table6 = np.load('training/feature_table6.npy')
table9 = np.load('training/feature_table9.npy')
table12 = np.load('training/feature_table12.npy')
table14 = np.load('training/feature_table14.npy')
table15 = np.load('training/feature_table15.npy')
table16 = np.load('training/feature_table16.npy')

In [4]:
full_table = np.vstack((table1, table2, table3, table4, table6, table9, table12, table14, table15, table16))

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [6]:
X = np.array(full_table[:, 1:-1], dtype=float)
y = [1 if i == 'True' else 0 for i in full_table[:, -1] ]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)

In [73]:
clf.predict_proba(X_train[:5])

array([[6.03641488e-03, 9.93963585e-01],
       [8.88178420e-16, 1.00000000e+00],
       [7.25096144e-01, 2.74903856e-01],
       [0.00000000e+00, 1.00000000e+00],
       [8.67503955e-01, 1.32496045e-01]])

In [74]:
y_train[:5]


[1, 1, 0, 0, 1]

In [75]:
clf.predict(X_train[:5])


array([1, 1, 0, 1, 0])

In [16]:
clf.score(X_train,y_train)

0.6711330512458862

In [13]:
clf.score(X_test, y_test)

0.6863988724453841

In [8]:
from sklearn.linear_model import SGDClassifier

In [18]:
sgd_clf = SGDClassifier(random_state=42, max_iter=1000, tol=1e-3)
sgd_clf.fit(X_train, y_train)

SGDClassifier(random_state=42)

In [19]:
sgd_clf.score(X_train, y_train)

0.4412317818523742

In [20]:
sgd_clf.score(X_test, y_test)

0.4439746300211416

In [82]:

test1 = np.load('testing/test1_table.npy')

In [84]:
res = sgd_clf.predict(test1)
res

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [46]:
np.unique(res, return_counts=True)

(array(['False'], dtype='<U5'), array([12]))

In [22]:
test2 = np.load('testing/test013.npy', allow_pickle=True)

In [27]:
results2 = []
for t in test2:
    print('-------')
    # res1 = sgd_clf.predict(t[1])
    res = clf.predict(t[1])
    # res = np.hstack((res1, res2))
    print(res)
    sum = np.sum(res)
    results2.append([sum, len(res), int(t[2]), sum/len(res)])
    print('likelihood genuine: ', round(sum/len(res) * 100),'%')
    print('likelihood forged: ', round((len(res)-sum)/len(res) * 100),'%')
    print("truth ", int(t[2]))

-------
[0 0 0 0 0 1 0 0 0 0 0 0]
likelihood genuine:  8 %
likelihood forged:  92 %
truth  0
-------
[1 0 1 1 1 0 1 1 1 0 1 1]
likelihood genuine:  75 %
likelihood forged:  25 %
truth  1
-------
[1 0 0 0 0 1 1 0 1 0 0 1]
likelihood genuine:  42 %
likelihood forged:  58 %
truth  0
-------
[0 0 0 0 1 0 1 0 0 1 0 0]
likelihood genuine:  25 %
likelihood forged:  75 %
truth  1
-------
[0 0 1 1 0 0 0 1 1 0 1 0]
likelihood genuine:  42 %
likelihood forged:  58 %
truth  1
-------
[1 1 0 1 1 1 1 0 0 1 0 0]
likelihood genuine:  58 %
likelihood forged:  42 %
truth  0
-------
[0 0 1 1 0 0 0 1 1 0 1 1]
likelihood genuine:  50 %
likelihood forged:  50 %
truth  0
-------
[0 0 1 1 0 0 0 1 1 0 0 1]
likelihood genuine:  42 %
likelihood forged:  58 %
truth  1
-------
[1 1 0 0 1 1 1 1 0 1 1 1]
likelihood genuine:  75 %
likelihood forged:  25 %
truth  1
-------
[1 0 0 1 1 1 1 1 1 1 1 1]
likelihood genuine:  83 %
likelihood forged:  17 %
truth  1
-------
[1 1 0 1 1 1 0 1 0 0 0 0]
likelihood genuine:  50 %
l

In [28]:
results2

[[1, 12, 0, 0.08333333333333333],
 [9, 12, 1, 0.75],
 [5, 12, 0, 0.4166666666666667],
 [3, 12, 1, 0.25],
 [5, 12, 1, 0.4166666666666667],
 [7, 12, 0, 0.5833333333333334],
 [6, 12, 0, 0.5],
 [5, 12, 1, 0.4166666666666667],
 [9, 12, 1, 0.75],
 [10, 12, 1, 0.8333333333333334],
 [6, 12, 1, 0.5],
 [9, 12, 0, 0.75],
 [3, 12, 1, 0.25],
 [8, 12, 0, 0.6666666666666666],
 [4, 12, 0, 0.3333333333333333],
 [6, 12, 1, 0.5],
 [10, 12, 1, 0.8333333333333334],
 [4, 12, 0, 0.3333333333333333],
 [6, 12, 0, 0.5],
 [9, 12, 1, 0.75],
 [2, 12, 0, 0.16666666666666666],
 [0, 12, 0, 0.0],
 [11, 12, 1, 0.9166666666666666],
 [4, 12, 0, 0.3333333333333333]]

In [37]:
tests = []
for file in os.listdir('testing'):
    if file.startswith('test_table_'):
        print(file)
        tests.append(np.load('testing/'+file, allow_pickle=True))



test_table_63.npy
test_table_62.npy
test_table_60.npy
test_table_61.npy
test_table_59.npy
test_table_58.npy
test_table_14.npy
test_table_15.npy
test_table_3.npy
test_table_16.npy
test_table_6.npy
test_table_13.npy
test_table_7.npy
test_table_5.npy
test_table_4.npy
test_table_35.npy
test_table_34.npy
test_table_8.npy
test_table_22.npy
test_table_23.npy
test_table_27.npy
test_table_32.npy
test_table_24.npy
test_table_30.npy
test_table_31.npy
test_table_25.npy
test_table_19.npy
test_table_42.npy
test_table_43.npy
test_table_41.npy
test_table_68.npy
test_table_50.npy
test_table_44.npy
test_table_51.npy
test_table_53.npy
test_table_52.npy
